In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql.functions import col
from pyspark.sql.types import *
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from pyspark.sql.functions import col, to_date, date_format, lit,expr
import random

In [3]:
spark = SparkSession.builder \
.master("local") \
.appName("test_v1") \
.getOrCreate()

In [4]:
raw_df = spark\
        .read\
        .csv('/mnt/group-14-pvc/data', header = True, inferSchema = True)

In [5]:
raw_df.show(5)

+-----------+-----------+----------------+----+--------------------+--------+--------+---------+
|supermarket|prices_(¬£)|prices_unit_(¬£)|unit|               names|    date|category|own_brand|
+-----------+-----------+----------------+----+--------------------+--------+--------+---------+
|      Sains|        2.5|             4.6|   l|Cornetto Classico...|20240129|  frozen|    false|
|      Sains|       1.19|            4.58|  kg|Sainsbury's Garli...|20240129|  frozen|     true|
|      Sains|        0.8|            0.94|  kg|Stamford Street C...|20240129|  frozen|    false|
|      Sains|        2.5|            7.31|  kg|Linda McCartney S...|20240129|  frozen|    false|
|      Sains|       5.25|            10.1|  kg|Sainsbury's Cod S...|20240129|  frozen|     true|
+-----------+-----------+----------------+----+--------------------+--------+--------+---------+
only showing top 5 rows



In [7]:
raw_df.count()

2104337

In [ ]:
columns_to_drop = ['prices_(£)', 'prices_unit_(£)', 'unit', 'own_brand']
raw_df = raw_df.drop(*columns_to_drop)

In [8]:
raw_df.printSchema()

root
 |-- supermarket: string (nullable = true)
 |-- prices_(¬£): double (nullable = true)
 |-- prices_unit_(¬£): double (nullable = true)
 |-- unit: string (nullable = true)
 |-- names: string (nullable = true)
 |-- date: string (nullable = true)
 |-- category: string (nullable = true)
 |-- own_brand: boolean (nullable = true)



In [7]:
raw_df = raw_df.withColumn("date", col("date").cast("string"))

raw_df = raw_df.withColumn("date", to_date(col("date"), "yyyyMMdd"))

In [12]:
raw_df.printSchema()

root
 |-- supermarket: string (nullable = true)
 |-- prices_(¬£): double (nullable = true)
 |-- prices_unit_(¬£): double (nullable = true)
 |-- unit: string (nullable = true)
 |-- names: string (nullable = true)
 |-- date: date (nullable = true)
 |-- category: string (nullable = true)
 |-- own_brand: boolean (nullable = true)



In [11]:
raw_df.show(5)

+-----------+-----------+----------------+----+--------------------+----------+--------+---------+
|supermarket|prices_(¬£)|prices_unit_(¬£)|unit|               names|      date|category|own_brand|
+-----------+-----------+----------------+----+--------------------+----------+--------+---------+
|      Sains|        2.5|             4.6|   l|Cornetto Classico...|2024-01-29|  frozen|    false|
|      Sains|       1.19|            4.58|  kg|Sainsbury's Garli...|2024-01-29|  frozen|     true|
|      Sains|        0.8|            0.94|  kg|Stamford Street C...|2024-01-29|  frozen|    false|
|      Sains|        2.5|            7.31|  kg|Linda McCartney S...|2024-01-29|  frozen|    false|
|      Sains|       5.25|            10.1|  kg|Sainsbury's Cod S...|2024-01-29|  frozen|     true|
+-----------+-----------+----------------+----+--------------------+----------+--------+---------+
only showing top 5 rows



In [13]:
raw_df.createOrReplaceTempView("transaction")

In [16]:

sql_query = """
    SELECT min(date) as Min_Date, max(date) as Max_Date
    FROM transaction
"""

result_df = spark.sql(sql_query)
result_df.show()

+----------+----------+
|  Min_Date|  Max_Date|
+----------+----------+
|2024-01-09|2024-01-29|
+----------+----------+



In [19]:
p_df = raw_df.drop('date')

In [20]:
p_df.printSchema()

root
 |-- supermarket: string (nullable = true)
 |-- prices_(¬£): double (nullable = true)
 |-- prices_unit_(¬£): double (nullable = true)
 |-- unit: string (nullable = true)
 |-- names: string (nullable = true)
 |-- category: string (nullable = true)
 |-- own_brand: boolean (nullable = true)



In [21]:
p_df = raw_df.withColumn("date", expr(
    "date_add(to_date('2023-01-01'), floor(rand() * 365))"))

In [22]:
p_df.show()

+-----------+-----------+----------------+----+--------------------+----------+--------+---------+
|supermarket|prices_(¬£)|prices_unit_(¬£)|unit|               names|      date|category|own_brand|
+-----------+-----------+----------------+----+--------------------+----------+--------+---------+
|      Sains|        2.5|             4.6|   l|Cornetto Classico...|2023-03-22|  frozen|    false|
|      Sains|       1.19|            4.58|  kg|Sainsbury's Garli...|2023-03-17|  frozen|     true|
|      Sains|        0.8|            0.94|  kg|Stamford Street C...|2023-12-31|  frozen|    false|
|      Sains|        2.5|            7.31|  kg|Linda McCartney S...|2023-04-20|  frozen|    false|
|      Sains|       5.25|            10.1|  kg|Sainsbury's Cod S...|2023-03-06|  frozen|     true|
|      Sains|        2.0|             5.0|  kg|Sainsbury‚Äôs Bee...|2023-12-10|  frozen|    false|
|      Sains|        5.5|            12.5|  kg|Young's Chip Shop...|2023-10-18|  frozen|    false|
|      Sai

In [23]:
p_df.printSchema()

root
 |-- supermarket: string (nullable = true)
 |-- prices_(¬£): double (nullable = true)
 |-- prices_unit_(¬£): double (nullable = true)
 |-- unit: string (nullable = true)
 |-- names: string (nullable = true)
 |-- date: date (nullable = true)
 |-- category: string (nullable = true)
 |-- own_brand: boolean (nullable = true)



In [24]:
p_df.createOrReplaceTempView("p_df")

In [25]:
sql_query = """
    SELECT min(date) as Min_Date, max(date) as Max_Date
    FROM p_df
"""

spark.sql(sql_query).show()

+----------+----------+
|  Min_Date|  Max_Date|
+----------+----------+
|2023-01-01|2023-12-31|
+----------+----------+



In [27]:
p_df.show(5)

+-----------+-----------+----------------+----+--------------------+----------+--------+---------+
|supermarket|prices_(¬£)|prices_unit_(¬£)|unit|               names|      date|category|own_brand|
+-----------+-----------+----------------+----+--------------------+----------+--------+---------+
|      Sains|        2.5|             4.6|   l|Cornetto Classico...|2023-03-22|  frozen|    false|
|      Sains|       1.19|            4.58|  kg|Sainsbury's Garli...|2023-03-17|  frozen|     true|
|      Sains|        0.8|            0.94|  kg|Stamford Street C...|2023-12-31|  frozen|    false|
|      Sains|        2.5|            7.31|  kg|Linda McCartney S...|2023-04-20|  frozen|    false|
|      Sains|       5.25|            10.1|  kg|Sainsbury's Cod S...|2023-03-06|  frozen|     true|
+-----------+-----------+----------------+----+--------------------+----------+--------+---------+
only showing top 5 rows



In [28]:
cities = ['Armagh', 'Bangor', 'Belfast', 'Lisburn', 'Londonderry', 'Newry', 'Bath', 'Birmingham', 'Bradford', 'Brighton & Hove', 'Bristol', 'Cambridge', 'Canterbury', 'Carlisle', 'Chelmsford', 'Chester', 'Chichester', 'Colchester', 'Coventry', 'Derby', 'Doncaster', 'Durham', 'Ely', 'Exeter', 'Gloucester', 'Hereford', 'Kingston-upon-Hull', 'Lancaster', 'Leeds', 'Leicester', 'Lichfield', 'Lincoln', 'Liverpool', 'London', 'Manchester', 'Milton Keynes', 'Newcastle-upon-Tyne', 'Norwich', 'Nottingham', 'Oxford', 'Peterborough', 'Plymouth', 'Portsmouth', 'Preston', 'Ripon', 'Salford', 'Salisbury', 'Sheffield', 'Southampton', 'Southend-on-Sea', 'St Albans', 'Stoke on Trent', 'Sunderland', 'Truro', 'Wakefield', 'Wells', 'Westminster', 'Winchester', 'Wolverhampton', 'Worcester', 'York', 'Aberdeen', 'Dundee', 'Dunfermline', 'Edinburgh', 'Glasgow', 'Inverness', 'Perth', 'Stirling', 'Bangor', 'Cardiff', 'Newport', 'St Asaph', 'St Davids', 'Swansea', 'Wrexham']


random.shuffle(cities)

p_df = p_df.withColumn("city", expr("element_at(array('{0}'), int(rand() * {1}) + 1)".format("','".join(cities), len(cities))))

In [29]:
p_df.show(5)

+-----------+-----------+----------------+----+--------------------+----------+--------+---------+---------------+
|supermarket|prices_(¬£)|prices_unit_(¬£)|unit|               names|      date|category|own_brand|           city|
+-----------+-----------+----------------+----+--------------------+----------+--------+---------+---------------+
|      Sains|        2.5|             4.6|   l|Cornetto Classico...|2023-03-22|  frozen|    false|       Plymouth|
|      Sains|       1.19|            4.58|  kg|Sainsbury's Garli...|2023-03-17|  frozen|     true|Brighton & Hove|
|      Sains|        0.8|            0.94|  kg|Stamford Street C...|2023-12-31|  frozen|    false|     Chelmsford|
|      Sains|        2.5|            7.31|  kg|Linda McCartney S...|2023-04-20|  frozen|    false|        Cardiff|
|      Sains|       5.25|            10.1|  kg|Sainsbury's Cod S...|2023-03-06|  frozen|     true|         Durham|
+-----------+-----------+----------------+----+--------------------+----------+-

In [30]:
p_df.createOrReplaceTempView("p_df")

In [32]:
sql_query = """
    SELECT supermarket, count(1) 
    FROM p_df 
    where city = 'Bath'
    group by supermarket
    order by count(1)
"""


# sql_query = """
#     SELECT city, count(1) 
#     FROM p_df group by city order by count(1)
# """

spark.sql(sql_query).show(80)

+-----------+--------+
|supermarket|count(1)|
+-----------+--------+
|       Aldi|    1343|
|  Morrisons|    5037|
|      Tesco|    6217|
|       ASDA|    6991|
|      Sains|    7694|
+-----------+--------+



In [75]:
sql_query = """
    SELECT count(distinct city)
    FROM p_df
"""

spark.sql(sql_query).show(80)

+--------------------+
|count(DISTINCT city)|
+--------------------+
|                  75|
+--------------------+



In [35]:
p_df = p_df.withColumnRenamed("prices_(¬£)", "price")\
       .withColumnRenamed("prices_unit_(¬£)", "price_unit")\
       .withColumnRenamed("names", "product_name")\
       .withColumnRenamed("category", "product_category")\
       .withColumnRenamed("unit", "measure_unit")
        

In [36]:
p_df.show()

+-----------+-----+----------+------------+--------------------+----------+----------------+---------+------------------+
|supermarket|price|price_unit|measure_unit|        product_name|      date|product_category|own_brand|              city|
+-----------+-----+----------+------------+--------------------+----------+----------------+---------+------------------+
|      Sains|  2.5|       4.6|           l|Cornetto Classico...|2023-03-22|          frozen|    false|          Plymouth|
|      Sains| 1.19|      4.58|          kg|Sainsbury's Garli...|2023-03-17|          frozen|     true|   Brighton & Hove|
|      Sains|  0.8|      0.94|          kg|Stamford Street C...|2023-12-31|          frozen|    false|        Chelmsford|
|      Sains|  2.5|      7.31|          kg|Linda McCartney S...|2023-04-20|          frozen|    false|           Cardiff|
|      Sains| 5.25|      10.1|          kg|Sainsbury's Cod S...|2023-03-06|          frozen|     true|            Durham|
|      Sains|  2.0|     

In [37]:
p_df.printSchema()

root
 |-- supermarket: string (nullable = true)
 |-- price: double (nullable = true)
 |-- price_unit: double (nullable = true)
 |-- measure_unit: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- date: date (nullable = true)
 |-- product_category: string (nullable = true)
 |-- own_brand: boolean (nullable = true)
 |-- city: string (nullable = true)



In [38]:
output_path = '/mnt/group-14-pvc/processed_data'
p_df.write.csv(output_path, header=True, mode="overwrite")

In [40]:
processed_df.count()

2104337